In [ ]:
#step1
pip install flask-ngrok flask-cors transformers --quiet
# Step 2: Install required packages
pip install flask flask-cors pyngrok transformers --quiet
# step 3
from pyngrok import conf
conf.get_default().auth_token = "2vIY6uW.......................6VSDEot"#paste yor auth token here.
#step 4 (final step)
# Step 1: Install Required Packages (Run this once in Colab)

# Step 2: Import Libraries
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
from transformers import RobertaTokenizer, T5ForConditionalGeneration
import torch

# Step 3: Load Your Model & Tokenizer
model_path = "/content/code_explainer_model"
tokenizer = RobertaTokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# Step 4: Setup Flask App with CORS
app = Flask(__name__)
CORS(app)

@app.route('/analyze', methods=['POST'])
def analyze():
    data = request.get_json()
    code = data.get("code", "")

    if not code.strip():
        return jsonify({"error": "Empty code"}), 400

    input_text = "explain and document: " + code
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=512)
    
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return jsonify({"result": result.strip()})

# Step 5: Run App with ngrok
port = 5000
public_url = ngrok.connect(port)
print(f"🔗 Public ngrok URL: {public_url}")
app.run(port=port)
